In [39]:
import numpy as np
import pandas as pd
import string
import warnings
warnings.filterwarnings('ignore')



Obtendo e tratando os dados

In [40]:
import pandas as pd
from urllib.request import Request, urlopen

url = 'https://www.fundamentus.com.br/resultado.php'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

req = Request(url, headers=header)
webpage = urlopen(req).read()

df = pd.read_html(webpage,  decimal=',', thousands='.')[0]

In [47]:
df
#df.shape

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
10,RRRP3,37.86,-8475.33,1.87,10.539,0.0000,1.406,3.57,30.84,5.34,...,14.23,0.3417,0.0220,4.79,0.0860,-0.0002,212615000.0,4.107890e+09,0.01,2.5632
11,GNDI3,66.95,-3584.82,5.84,3.423,0.0281,2.378,79.65,87.05,-6.21,...,50.88,0.0393,-0.0009,1.17,0.0309,-0.0016,261553000.0,7.116080e+09,0.53,0.2523
13,AALR3,17.70,-372.23,1.78,1.842,0.0000,0.824,-20.89,22.25,-2.48,...,12.84,0.0828,0.0017,0.83,0.0402,-0.0048,9274900.0,1.175290e+09,0.66,-0.0042
14,DASA3,23.41,-362.24,1.77,1.316,0.0126,0.714,16.06,83.25,-2.49,...,15.44,0.0158,-0.0026,1.17,0.0104,-0.0049,16207800.0,7.418660e+09,0.72,0.3037
15,CASH3,2.15,-360.65,2.42,8.293,0.0027,2.091,2.92,-190.11,3.30,...,-193.15,-0.0436,-0.0115,21.09,-0.0363,-0.0067,111604000.0,7.138490e+08,0.00,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
973,HAPV3,11.71,214.79,7.88,8.751,0.0017,4.806,188.62,160.07,-27.21,...,57.63,0.0547,0.0413,1.14,0.0356,0.0367,491565000.0,1.061560e+10,0.19,0.3743
977,BIDI3,5.85,435.66,1.78,0.000,0.0044,0.000,0.00,0.00,0.00,...,0.00,0.0000,0.0000,0.00,0.0000,0.0041,12529300.0,8.488640e+09,0.00,0.4469
978,BIDI11,17.96,445.84,1.82,0.000,0.0043,0.000,0.00,0.00,0.00,...,0.00,0.0000,0.0000,0.00,0.0000,0.0041,379222000.0,8.488640e+09,0.00,0.4469
979,BIDI4,6.12,455.77,1.86,0.000,0.0042,0.000,0.00,0.00,0.00,...,0.00,0.0000,0.0000,0.00,0.0000,0.0041,98570600.0,8.488640e+09,0.00,0.4469


In [42]:
for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  df[coluna] = df[coluna].str.replace('.', '')
  df[coluna] = df[coluna].str.replace(',', '.')
  df[coluna] = df[coluna].str.rstrip('%').astype('float') / 100


Analisando os Dados

In [45]:
#filtro para tirar empresas muito pequenas
df = df[df['Liq.2meses'] > 1000000]  #subsituindo no dataframe original


,Posição,EV/EBIT,ROIC
0,1,PSSA3,BRBI11
1,2,WEST3,CMIN3
2,3,GOAU3,PSSA3
3,4,GOAU4,UNIP6
4,5,SYNE3,UNIP3
...,...,...,...
145,146,MDIA3,CIEL3
146,147,ALSO3,MULT3
147,148,IGTI3,CYRE3
148,149,TOTS3,DIRR3


In [52]:
ranking = pd.DataFrame()
ranking['Posição'] = range(1,151)
ranking['EV/EBIT'] = df[df['EV/EBIT'] > 0].sort_values(by=['EV/EBIT'])['Papel'][:150].values
ranking['ROIC'] = df.sort_values(by=['ROIC'], ascending=False)['Papel'][:150].values

In [55]:
a = ranking.pivot_table(columns='EV/EBIT', values='Posição')

b = ranking.pivot_table(columns='ROIC', values='Posição')

In [59]:
t = pd.concat([a,b]) #Ranking da empresa por EV/EBIT E ROIC
t

,ABEV3,AGRO3,AGXY3,ALLD3,ALSO3,ALUP11,AMAR3,ASAI3,AURA33,B3SA3,...,MOSI3,MULT3,PETZ3,RADL3,RDOR3,RRRP3,TFCO4,VAMO3,VIVA3,WEGE3
Posição,133,29,136.0,23,147.0,32,60,78,27,127,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Posição,63,20,NaN,48,NaN,67,108,35,7,50,...,138.0,147.0,133.0,93.0,118.0,132.0,30.0,81.0,68.0,33.0


In [60]:
rank = t.dropna(axis=1).sum()  #dropna elimina todas as linhas com valores faltantes      #nesse caso o dropna vai agir nas linhas do DataFrame
rank                           #axis=1 significa que a função é aplicada em função da linha no DataFrame

ABEV3     196
AGRO3      49
ALLD3      71
ALUP11     99
AMAR3     168
         ... 
VITT3     171
VLID3     231
VULC3     201
VVEO3     207
WIZS3      60
Length: 131, dtype: int64

In [62]:
rank.sort_values()[:15]    #15 melhores empresas classificadas 

PSSA3     4
SYNE3    17
CMIN3    20
BRKM5    21
ETER3    24
VALE3    25
USIM5    26
USIM3    26
GOAU4    27
PLPL3    27
GOAU3    27
CSNA3    29
MRFG3    30
TASA4    32
GGBR3    34
dtype: int64